# Deep Reinforcement Learning with Stable Baselines 3

In this exercise, you'll train a walking behavior using stable baselines, a library made for deep reinforcement learning.

First, let's import the necessary parts of the libraries for this exercise.

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

**Important:** should this step yield any errors, check if this notebook runs in the correct conda environment. Without the imports working, none of the following steps will work!

Next, we need to create the environments for which we want to learn a walking behavior. The goal of this environment is to make the robot move to the right side (positive x direction). For more information about the HalfCheetah environment, visit https://gymnasium.farama.org/environments/mujoco/half_cheetah/.

In [ ]:
env_id = "HalfCheetah-v4"
env_raw = gym.make(env_id, render_mode="rgb_array")
# A seperate environment is required for evaluation
eval_env_raw = gym.make(env_id, render_mode="rgb_array")

With Deep Reinforcement Learning, an agent tries to maximize the reward it gets for the actions he performs. Your task is now to write a function to calculate the reward for the agent.

The following are the most important values that you can use for calculating the reward.
- `action` is an array containing the actions for the agent. The documentation linked above explains what these values represent.
- Use `self.env.data.xpos` to get the positions of the robot's body parts. As calling the "inner" environment's `step` method advances the simulation using the generated action, it makes a difference if you obtain those positions before calling `step` or after. This is a two-dimensional environment. Therefore, `self.env.data.xpos[?][1]` (the y-coordinate) is of no interest for all body parts. The following information is taken from the definition of the environment: https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/mujoco/assets/half_cheetah.xml. The body parts should usually appear in the same order in the array as defined in the XML file using `<body>` tags.
  * `self.env.data.xpos[1]`: torso
  * `self.env.data.xpos[2]`: back thigh
  * `self.env.data.xpos[3]`: back shin
  * `self.env.data.xpos[4]`: back foot
  * `self.env.data.xpos[5]`: front thigh
  * `self.env.data.xpos[6]`: front shin
  * `self.env.data.xpos[7]`: front foot
- `self.dt` contains the time elapsed during the simulation step.

In [ ]:
class HalfCheetahRewardWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def step(self, action):
        x_position_torso_before = self.env.data.xpos[1][0]
        
        # Run the simulation and apply the action
        observation, _, terminated, truncated, info = self.env.step(action)
        
        x_position_torso_after = self.env.data.xpos[1][0]
        
        # TODO: calculate the reward here
        reward = 0
        
        return observation, reward, terminated, truncated, info

# Wrap the existing environments
env = HalfCheetahRewardWrapper(env_raw)
eval_env = Monitor(HalfCheetahRewardWrapper(eval_env_raw))

Now, we need to create the model. It is initialized with a random policy.
For this exercise, we'll use the PPO (Proximal Policy Optimization) algorithm.

In [ ]:
model = PPO("MlpPolicy", env, verbose=1)

Let's see how the random policy performs:

In [ ]:
mean_reward, std_deviation = evaluate_policy(model, eval_env, n_eval_episodes=100)
print("mean reward:", mean_reward)
print("standard deviation:", std_deviation)

Now, it's time for learning how to walk. You can adjust the number of timesteps to learn on based on the strength of your computer. More timesteps take longer but may improve the resulting behavior.

In [ ]:
model.learn(total_timesteps=200000)

Another evaluation is required to measure if an improvement was made to the model compared to its initial random state.

In [ ]:
mean_reward, std_deviation = evaluate_policy(model, eval_env, n_eval_episodes=100)
print("mean reward:", mean_reward)
print("standard deviation:", std_deviation)

You can export a GIF of your trained behavior using the following code.

In [ ]:
import imageio
import numpy as np

filename = "halfcheetah.gif"

images = []
obs = model.env.reset()
img = model.env.render()
start_pos = model.env.envs[0].data.xpos[1][0]
for _ in range(200):
    images.append(img)
    action, _ = model.predict(obs)
    obs, _, _, _ = model.env.step(action)
    img = model.env.render()

end_pos = model.env.envs[0].data.xpos[1][0]
print(f"Distance travelled: {end_pos - start_pos}m")

imageio.mimsave(filename, [np.array(img) for img in images], duration=50)

Within the visualization, one side of a (black or white) square is $\frac{2}{3}$ meters long.

If you are interested in training robots in other environments, check out the already existing environments in Gymnasium: https://gymnasium.farama.org/environments/mujoco/.
You should be able to use another environment id and everything else in the notebook should mostly continue to work. Of course, you then need to find out yourself what the positions in `self.env.data.xpos` represent for your chosen environment.
Note: as of writing this (August 2023), there seem to be some problems with the Hopper and the Walker2D environment that prevent these environments from being used.

Now let's see how high the robot is able to jump. Once again, we need to define a reward function.
It might make sense to manually set `terminated` and `truncated` in this case.
Have a look at https://gymnasium.farama.org/api/env/#gymnasium.Env.step for an explanation of their meaning.

In [ ]:
class HalfCheetahJumpRewardWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def step(self, action):
        # Run the simulation and apply the action
        observation, _, terminated, truncated, info = self.env.step(action)
        
        # TODO: calculate the reward here
        reward = 0
        
        return observation, reward, terminated, truncated, info

# Wrap the existing environments
jump_env = HalfCheetahJumpRewardWrapper(env_raw)
jump_eval_env = Monitor(HalfCheetahJumpRewardWrapper(eval_env_raw))

Again, we need to create a new model.

In [ ]:
jump_model = PPO("MlpPolicy", jump_env, verbose=1)

Let's have a view at performance of the random policy:

In [ ]:
mean_reward, std_deviation = evaluate_policy(jump_model, jump_eval_env, n_eval_episodes=100)
print("mean reward:", mean_reward)
print("standard deviation:", std_deviation)

Train the network:

In [ ]:
jump_model.learn(total_timesteps=200000)

Evaluate the result:

In [ ]:
mean_reward, std_deviation = evaluate_policy(jump_model, jump_eval_env, n_eval_episodes=100)
print("mean reward:", mean_reward)
print("standard deviation:", std_deviation)

Of course, you can export another GIF of your trained jumping behavior using the following code.

In [ ]:
import imageio
import numpy as np

filename = "halfcheetah_jump.gif"

images = []
obs = jump_model.env.reset()
img = jump_model.env.render()
for _ in range(100):
    images.append(img)
    action, _ = jump_model.predict(obs)
    obs, _, _, _ = jump_model.env.step(action)
    img = jump_model.env.render()

imageio.mimsave(filename, [np.array(img) for img in images], duration=50)